In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import matplotlib.pyplot as plt

import os

import random
import cv2

2023-03-15 21:28:35.275365: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import pathlib
ds_path = "/Users/mantesssa/Projects/nn/nn_tf/BirdVsDrone/"
data_dir = pathlib.Path(ds_path)

image_count = len(list(data_dir.glob('*/*.*')))
print(image_count)

829


In [3]:
dirname = "/Users/mantesssa/Projects/nn/nn_tf/BirdVsDrone/"
tree = os.walk(dirname)

fulllist =[]
for i in tree:
    fulllist.append(i)
    
listofdronesnames = []
for i in fulllist[1][2]:
#     if i[-4:] !='webp':
    if i[-4:] =='jpeg' or i[-4:] =='JPEG' or i[-3:] =='png':
        listofdronesnames.append(i) 
listofdroneslabels = [0]*len(listofdronesnames)

listofbirdsnames = []
for i in fulllist[2][2]:
    if i[-4:] =='jpeg' or i[-4:] =='JPEG' or i[-3:] =='png':
        listofbirdsnames.append(i) # .DS_Store
listofbirdslabels = [1]*len(listofbirdsnames)



fulllistnames = listofdronesnames + listofbirdsnames
fulllistlabels = listofdroneslabels + listofbirdslabels
# print(len(fulllistnames))
# print(fulllistlabels)



listofindexes = []
for i in range(len(fulllistlabels)): # len = 829
    listofindexes.append(i)

random.shuffle(listofindexes)
shuffledlistofindexes = listofindexes


len(shuffledlistofindexes)

826

In [17]:
# dataset = 800 indexes from 0 to 799

# 0|    0-79
# 1|    80-159
# 2|    160-239
# 3|    240-319
# 4|    320-399
# 5|    400-479
# 6|    480-559
# 7|    560-639
# 8|    640-719
# 9|    720-799

# 0|    0-39
# 1|    40-79
# 2|    80-119 
# 3|   120-159
# 4|   160-199
# 5|   200-239
# 6|   240-279
# 7|   280-319
# 8|   320-359
# 9|   360-399
# 10|  400-439
# 11|  440-479
# 12|  480-519
# 13|  520-559
# 14|  560-599
# 15|  600-639
# 16|  640-679
# 17|  680-719
# 18|  720-759
# 19|  760-799





# (0) 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
# 0 (1) 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
# 0 1 (2) 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
# 0 1 2 (3) 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
# 0 1 2 3 (4) 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
# 0 1 2 3 4 (5) 6 7 8 9 10 11 12 13 14 15 16 17 18 19
# 0 1 2 3 4 5 (6) 7 8 9 10 11 12 13 14 15 16 17 18 19
# 0 1 2 3 4 5 6 (7) 8 9 10 11 12 13 14 15 16 17 18 19
# 0 1 2 3 4 5 6 7 (8) 9 10 11 12 13 14 15 16 17 18 19
# 0 1 2 3 4 5 6 7 8 (9) 10 11 12 13 14 15 16 17 18 19
# 0 1 2 3 4 5 6 7 8 9 (10) 11 12 13 14 15 16 17 18 19
# 0 1 2 3 4 5 6 7 8 9 10 (11) 12 13 14 15 16 17 18 19
# 0 1 2 3 4 5 6 7 8 9 10 11 (12) 13 14 15 16 17 18 19
# 0 1 2 3 4 5 6 7 8 9 10 11 12 (13) 14 15 16 17 18 19
# 0 1 2 3 4 5 6 7 8 9 10 11 12 13 (14) 15 16 17 18 19
# 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 (15) 16 17 18 19
# 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 (16) 17 18 19
# 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 (17) 18 19
# 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 (18) 19
# 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 (19)


class DataGenerator(tf.keras.utils.Sequence):
#     'Generates data for Keras'                      shufled list of indexes
    def __init__(self, fulllistnames, fulllistlabels, list_IDs, is_validation=False, 
                 batch_size=40, dim=(None, 180, 180, 3), n_channels=1, n_classes=2):
        'Initialization'
        self.fulllistnames = fulllistnames
        self.fulllistlabels = fulllistlabels
        self.list_IDs = list_IDs
        self.is_validation = is_validation
        self.batch_size = batch_size
        self.dim = dim
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.valid = is_validation
        self.indexes = np.arange(len(self.list_IDs))
        self.valid_index = 0
        self.sval = False

        
    def __len__(self):
        'Denotes the number of batches per epoch'
        # print(int(np.floor(len(self.list_IDs) / self.batch_size))-2)
        return int(np.floor(len(self.list_IDs) / self.batch_size))-2 # = 18 
        # return int(np.floor(len(self.list_IDs) / self.batch_size))- # = 8


    
    
    def __getitem__(self, index): # index = number of batch from 0 to __len__ 
        if self.is_validation:    #if validation set                         WHY CALLED MANY TIMES???
            if self.sval:
                indexes = self.indexes[(self.valid_index+1)*self.batch_size : (self.valid_index+2)*self.batch_size]
            else:
                indexes = self.indexes[self.valid_index*self.batch_size : (self.valid_index+1)*self.batch_size]
                self.sval = True
            # print(" i=", self.valid_index, indexes[0],indexes[-1])        
            list_IDs_temp = [self.list_IDs[k] for k in indexes]
#             print("if valid list_IDs_temp=", len(list_IDs_temp))
#             print("self.valid_index",self.valid_index)
#             print("if valid list_IDs_temp=", len(list_IDs_temp))
            
            X, y = self.__data_generation(list_IDs_temp)
        else: #if test set 
            if index >= self.valid_index:
                newindex = index+2
            else:
                newindex = index
            indexes = self.indexes[newindex*self.batch_size:(newindex+1)*self.batch_size]
            # print(" tr> i=", newindex, indexes[0],"-",indexes[-1],"ind=",index,"<=",self.valid_index)
            list_IDs_temp = [self.list_IDs[k] for k in indexes]
#             print("if training list_IDs_temp=", len(list_IDs_temp))
#             print("newindex",newindex)
            X, y = self.__data_generation(list_IDs_temp)
        return X, y

    def on_epoch_end(self):
        print("\n valid_index =", self.valid_index)
        self.valid_index += 2 # ???

        # if self.valid_index == 19:
        #     self.valid_index = 0
        # else:
        #     self.valid_index += 2 # ???

        

        
    def __data_generation(self, list_IDs_temp):
        y = np.empty((self.batch_size), dtype=int)
        X = []
        for i, ID in enumerate(list_IDs_temp):
            named= self.fulllistnames[ID]
            classnumber= self.fulllistlabels[ID]
            if classnumber == 0:
                classnamed = "Drones"
            else:
                classnamed = "Birds"
            img = cv2.imread("/Users/mantesssa/Projects/nn/nn_tf/BirdVsDrone/" + classnamed + "/" + named)
#             print("/Users/mantesssa/Projects/nn/nn_tf/BirdVsDrone/" + classnamed + "/" + named)
            RGB_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            smallimgb = cv2.resize(RGB_img, (180,180),interpolation=cv2.INTER_CUBIC)
            # plt.imshow(smallimgb)
            X.append(smallimgb)
            y[i] = classnumber
#         print(y.shape)
#         print(np.array(keras.utils.to_categorical(y, num_classes=self.n_classes)).shape)

        return np.array(X), keras.utils.to_categorical(y, num_classes=self.n_classes)
        





In [18]:
batch_size = 40
img_height = 180
img_width = 180
class_names = ['Birds', 'Drones']
num_classes = len(class_names)

In [19]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.12),
    layers.RandomZoom(0.3),
  ]
)

model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, name="outputs")
])

# model.summary()

In [20]:
STEPS_PER_EPOCH = 10
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=STEPS_PER_EPOCH*100,
  decay_rate=1,
  staircase=False)



def get_optimizer():
  return tf.keras.optimizers.Adam(lr_schedule)## Compile and train the model

In [21]:
# model.compile(optimizer=get_optimizer(),
model.compile(optimizer='adam',
              # loss='binary_crossentropy',
              loss='categorical_crossentropy',
              
              metrics=['accuracy'])

In [22]:
# Generators
#     def __init__(self, fulllistnames, fulllistlabels, list_IDs, is_validation=False, 
#                  batch_size=80, dim=(None, 180, 180, 3), n_channels=1, n_classes=2):

params = {'dim': (None, 180, 180, 3),
          'batch_size': 40,
          'n_classes': 2,
          'n_channels': 1,}

training_generator = DataGenerator(fulllistnames,fulllistlabels,shuffledlistofindexes,False, **params)
validation_generator = DataGenerator(fulllistnames,fulllistlabels,shuffledlistofindexes,True, **params)
# print(shuffledlistofindexes)

print(training_generator)
print(validation_generator)

# Train model on dataset

# epochs = 10 # n*10 # Do not work
# history = model.fit(training_generator,
#                     validation_data=validation_generator,
#                     epochs=epochs,
#                     use_multiprocessing=True,
#                     workers=6)



epochs = 20 # n*10
history = model.fit(training_generator,
          validation_data=validation_generator,
         epochs=epochs)




 tr> i= 2 80 - 119 ind= 0 <= 0
Epoch 1/20
 tr> i= 4 160 - 199 ind= 2 <= 0
 tr> i= 9 360 - 399 ind= 7 <= 0
18/18 [==============================] - ETA: 0s - loss: 8.1101 - accuracy: 0.5056 i= 0 0 39
 i= 0 0 39
 i= 0 0 39
 i= 0 0 39
 i= 0 0 39
 i= 0 0 39
 i= 0 0 39
 i= 0 0 39
 i= 0 0 39
 i= 0 0 39
 i= 0 0 39
 i= 0 0 39
 i= 0 0 39
 i= 0 0 39
 i= 0 0 39
 i= 0 0 39
 i= 0 0 39
 i= 0 0 39
 i= 0 0 39

 valid_index = 0
18/18 [==============================] - 33s 2s/step - loss: 8.1101 - accuracy: 0.5056 - val_loss: 10.4768 - val_accuracy: 0.6500

 valid_index = 0
Epoch 2/20
 tr> i= 17 680 - 719 ind= 15 <= 2
 tr> i= 15 600 - 639 ind= 13 <= 2
18/18 [==============================] - ETA: 0s - loss: 8.5068 - accuracy: 0.5278 i= 2 80 119
 i= 2 80 119
 i= 2 80 119
 i= 2 80 119
 i= 2 80 119
 i= 2 80 119
 i= 2 80 119
 i= 2 80 119
 i= 2 80 119
 i= 2 80 119
 i= 2 80 119
 i= 2 80 119
 i= 2 80 119
 i= 2 80 119
 i= 2 80 119
 i= 2 80 119
 i= 2 80 119
 i= 2 80 119

 valid_index = 2
18/18 [=================

2023-03-15 21:43:21.644381: W tensorflow/core/framework/op_kernel.cc:1768] UNKNOWN: IndexError: index 4575657221408423936 is out of bounds for axis 1 with size 2
Traceback (most recent call last):

  File "/Users/mantesssa/.pyenv/versions/anaconda3-2022.10/envs/tf/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 271, in __call__
    ret = func(*args)

  File "/Users/mantesssa/.pyenv/versions/anaconda3-2022.10/envs/tf/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/Users/mantesssa/.pyenv/versions/anaconda3-2022.10/envs/tf/lib/python3.10/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 1035, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/Users/mantesssa/.pyenv/versions/anaconda3-2022.10/envs/tf/lib/python3.10/site-packages/keras/engine/data_adapter.py", line 903, in wrapped_generator
    for data in generator_fn():

  Fil

UnknownError: Graph execution error:

IndexError: index 4575657221408423936 is out of bounds for axis 1 with size 2
Traceback (most recent call last):

  File "/Users/mantesssa/.pyenv/versions/anaconda3-2022.10/envs/tf/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 271, in __call__
    ret = func(*args)

  File "/Users/mantesssa/.pyenv/versions/anaconda3-2022.10/envs/tf/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/Users/mantesssa/.pyenv/versions/anaconda3-2022.10/envs/tf/lib/python3.10/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 1035, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/Users/mantesssa/.pyenv/versions/anaconda3-2022.10/envs/tf/lib/python3.10/site-packages/keras/engine/data_adapter.py", line 903, in wrapped_generator
    for data in generator_fn():

  File "/Users/mantesssa/.pyenv/versions/anaconda3-